In [2]:
#!pip install seaborn
#!pip install xlrd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#Sla de excel op als een csv

# Read the Excel file using xlrd
df = pd.read_csv('files/titanic3.csv', sep=',')
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


Perform data preprocessing, which includes handling missing values, converting categorical features to numerical, and feature scaling.

In [4]:
sub_df = df.groupby('sex')
df.head()
#df.dtypes
df.age.value_counts
#df.body.value_counts
#df.survived.value_counts
#sub_df.head()
#sub_df.sex.value_counts

<bound method IndexOpsMixin.value_counts of 0       29.0000
1        0.9167
2        2.0000
3       30.0000
4       25.0000
         ...   
1305        NaN
1306    26.5000
1307    27.0000
1308    29.0000
1309        NaN
Name: age, Length: 1310, dtype: float64>

In [5]:
df.describe()

,pclass,survived,age,sibsp,parch,fare,body
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,121.000000
mean,2.294882,0.381971,29.881135,0.498854,0.385027,33.295479,160.809917
std,0.837836,0.486055,14.413500,1.041658,0.865560,51.758668,97.696922
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800,72.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,155.000000
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000,256.000000
max,3.000000,1.000000,80.000000,8.000000,9.000000,512.329200,328.000000


In [13]:
# Ceil the values in the "Age" column
df['age'] = np.ceil(df['age']).copy()

# Display the modified DataFrame
print(df)

#Geen idee wa ik met de NaN moet aanvangen bij age

      pclass  survived                                             name  \
0        1.0       1.0                    Allen, Miss. Elisabeth Walton   
1        1.0       1.0                   Allison, Master. Hudson Trevor   
2        1.0       0.0                     Allison, Miss. Helen Loraine   
3        1.0       0.0             Allison, Mr. Hudson Joshua Creighton   
4        1.0       0.0  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)   
...      ...       ...                                              ...   
1305     3.0       0.0                            Zabour, Miss. Thamine   
1306     3.0       0.0                        Zakarian, Mr. Mapriededer   
1307     3.0       0.0                              Zakarian, Mr. Ortin   
1308     3.0       0.0                               Zimmerman, Mr. Leo   
1309     NaN       NaN                                              NaN   

         sex   age  sibsp  parch  ticket      fare    cabin embarked boat  \
0     female  29.0    

Hoe vervangen we NaN in body en in boat? Want boat is volgens mij de nummer van de boot waar ze opzitten en body kan in principe vervangen worden door avg maar hoe beinvloed da het model?

Vanaf nu komen de selecties

1. Target Variable:

 * "Survived" will be your target variable, as you want to predict survival. 
 
2. Predictor Variables (Features):

* "Pclass": The passenger class can be an important predictor. Higher classes may have had a higher priority for lifeboats.
* "Sex": Gender could be a significant factor as well, as women and children were given priority during the evacuation.
* "Age": Age may be important, as children and the elderly might have had different survival rates.
* "SibSp" (Number of Siblings/Spouses Aboard): This might provide insights into family groups.
* "Parch" (Number of Parents/Children Aboard): This might provide insights into family groups.
* "Fare": Fare paid may also be related to passenger class and could be indicative of socio-economic status.
* "Embarked": The port of embarkation may not be directly related to survival but could have some indirect effects.


3. Irrelevant Variables:

* "Name": Passenger names are unlikely to be directly related to survival.
* "Ticket": Ticket numbers are unlikely to be directly related to survival.
* "Cabin": Cabin numbers might contain information, but there are many missing values, and it can be challenging to extract meaningful patterns.
* "Boat" and "Body": These columns appear to contain post-incident information and are unlikely to be predictive.
* "Home Dest": Passenger's home destinations may not be directly related to survival.

In [11]:
# Select relevant columns
selected_columns = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'survived']

# Create a new DataFrame with only the selected columns
titanic_df = df[selected_columns]

# Now, 'titanic_df' contains only the relevant columns for your analysis
print(titanic_df)

      pclass     sex   age  sibsp  parch      fare embarked  survived
0        1.0  female  29.0    0.0    0.0  211.3375        S       1.0
1        1.0    male   1.0    1.0    2.0  151.5500        S       1.0
2        1.0  female   2.0    1.0    2.0  151.5500        S       0.0
3        1.0    male  30.0    1.0    2.0  151.5500        S       0.0
4        1.0  female  25.0    1.0    2.0  151.5500        S       0.0
...      ...     ...   ...    ...    ...       ...      ...       ...
1305     3.0  female   NaN    1.0    0.0   14.4542        C       0.0
1306     3.0    male  27.0    0.0    0.0    7.2250        C       0.0
1307     3.0    male  27.0    0.0    0.0    7.2250        C       0.0
1308     3.0    male  29.0    0.0    0.0    7.8750        S       0.0
1309     NaN     NaN   NaN    NaN    NaN       NaN      NaN       NaN

[1310 rows x 8 columns]
